In [1]:
import numpy as np
import cv2 as cv

In [2]:
# Function to read the secret image
def read_image(image_name):
    image = cv.imread(image_name)
    print("The shape of the secret image is = ", image.shape)
    resized_image = cv.resize(image, (500,500))
    cv.imshow("Secret Image",resized_image)
    cv.waitKey(0)
    cv.imwrite("test.png", image)
    return image

In [3]:
# Function to generate a random key
def generateRandomKey(key_shape):
    dealer_key = np.zeros(key_shape).astype(int)
    for i in range(0, len(dealer_key)):
        for j in range(0, len(dealer_key[i])):
            for k in range(0, 3):
                dealer_key[i][j][k] = (np.random.randint(0,255))
    return dealer_key


In [4]:
# Function to convert the secret image into encrypted image
def convertToEncryptedImage(image, random_key):
    print("Converting to Encrypted Image...")
    encrypted_image = np.zeros(image.shape).astype(int)
    
    for i in range(0, len(random_key)):
        for j in range(0, len(random_key[i])):
            for k in range(0, 3):
                encrypted_image[i][j][k] = image[i][j][k]^random_key[i][j][k]
    
    return encrypted_image

In [5]:
# Function to generate unique id's for each participant
def getUniqueIds(n):
    temp_arr = np.zeros(n).astype(int)
    for i in range(n):
        temp_arr[i] = np.random.randint(0,255)
    return temp_arr

In [6]:
#Function to get encrypted id's for each participant
def getEncryptedIds(unique_ids, n):
    temp_arr = []
    for i in range(n):
        x = np.zeros(8).astype(int)
        s = '{0:08b}'.format(unique_ids[i])
        msb = s[0:4]
        lsb = s[4:8]
        for i in range(4):
            x[i] = int(msb[i])
        for i in range(4):
            x[i+4] = int(msb[i])^int(lsb[i])
        res = int("".join(str(l) for l in x), 2)
        temp_arr.append(res)
        
    return temp_arr

In [7]:
# Left circular rotation of an array str -> x times
def leftRotate(bit_8_s, x):
    temp_str = []
    for i in range(x):
        temp_str.append(bit_8_s[i])
    check_point = 0
    for i in range(0, len(bit_8_s)-x):
        bit_8_s[i] = bit_8_s[i+x]
        check_point = i
    for i in range(0,x):
        bit_8_s[check_point+1+i] = temp_str[i]
    return bit_8_s

In [8]:
#Function to perform left circular shift
def leftCircularShift(R1, x):

    for i in range(0, len(R1)):
        for j in range(0, len(R1[i])):
            for k in range(0, 3):
                s = '{0:08b}'.format(R1[i][j][k])
                temp_s = np.zeros(8).astype(int)
                for l in range(8):
                    temp_s[l] = int(s[l])
                x = x%8
                temp_s = leftRotate(temp_s, x)
                res = int("".join(str(l) for l in temp_s), 2)
                R1[i][j][k] = res
    
    return R1

In [9]:
# Right circular rotation of an array str -> x times
def rightRotate(bit_8_s, x):
    temp_str = []
    for i in range(len(bit_8_s)-x,len(bit_8_s)):
        temp_str.append(bit_8_s[i])
    for i in range(len(bit_8_s)-1, x-1, -1):
        bit_8_s[i] = bit_8_s[i-x]
    for i in range(0,x):
        bit_8_s[i] = temp_str[i]
    return bit_8_s

In [10]:
#Function to perform right circular shift
def rightCircularShift(R1, x):

    for i in range(0, len(R1)):
        for j in range(0, len(R1[i])):
            for k in range(0, 3):
                s = '{0:08b}'.format(R1[i][j][k])
                temp_s = np.zeros(8).astype(int)
                for l in range(8):
                    temp_s[l] = int(s[l])
                x = x%8
                temp_s = rightRotate(temp_s, x)
                res = int("".join(str(l) for l in temp_s), 2)
                R1[i][j][k] = res
    
    return R1

In [11]:
N = input('Enter the total number of shares (N) : ')
N = int(N)
print("The total number of shares (N) = ", N)
K = input("Enter the number of shares required to retreive the secret image (K) :")
K = int(K)
print("The number of shares required to retreive the secret image (K) = ", K)
image_name = input("Enter the name of the image (with extension)")
image = read_image(image_name)
random_key = generateRandomKey(image.shape)
print("The shape of random key = ", random_key.shape)
encrypted_image = convertToEncryptedImage(image, random_key)
print("The shape of encrypted image = ", encrypted_image.shape)
unique_ids = getUniqueIds(N)
print("The unique id's of all the N participants are :")
print(unique_ids)

The total number of shares (N) =  7
The number of shares required to retreive the secret image (K) =  3
The shape of the secret image is =  (1225, 840, 3)
The shape of random key =  (1225, 840, 3)
Converting to Encrypted Image...
The shape of encrypted image =  (1225, 840, 3)
The unique id's of all the N participants are :
[ 96 224 203 148 108 240 244]


In [12]:
# Function to get authenticated image for each participant (R1)
def getR1(encrypted_image, shares):
    R1 = np.zeros(encrypted_image.shape).astype(int)
    R1_remainder = np.zeros(encrypted_image.shape).astype(int)
    
    for i in range(0, len(R1)):
        for j in range(0, len(R1[i])):
            for k in range(0, 3):
                R1[i][j][k] = int(encrypted_image[i][j][k]//int(shares))
                R1_remainder[i][j][k] = encrypted_image[i][j][k]%int(shares)
    
    return [R1, R1_remainder]

authenticated_image = getR1(encrypted_image, K) # array of R1 and R1_remainder 
print(authenticated_image[0].shape, authenticated_image[1].shape)

(1225, 840, 3) (1225, 840, 3)


In [13]:
encrypted_ids = getEncryptedIds(unique_ids, N)
print("The encrypted ids are : ")
print(encrypted_ids)

The encrypted ids are : 
[102, 238, 199, 157, 106, 255, 251]


In [14]:
#generating n shares
def generate_N_shares(R1, n, encrypted_ids):
    S = []
    for i in range(0,n):
        temp = R1
        temp_S = leftCircularShift(temp, encrypted_ids[i])
        S.append(temp_S)
    return S


In [15]:

shares = generate_N_shares(authenticated_image[0], N, encrypted_ids)


In [16]:
# Function to encode the 8 bit encrypted ids into each of the corresponding shares
def encode_shares(S, n, encrypted_ids):
    for l in range(0,n):
        encrypted_string = '{0:08b}'.format(encrypted_ids[l])
        count_bits = 0
        for i in range(0, len(S[l])):
            for j in range(0, len(S[l][i])):
                for k in range(0, 3):
                    temp_string = '{0:08b}'.format(S[l][i][j][k])
                    temp_array = np.zeros(8).astype(int)
                    for b in range(8):
                        temp_array[b] = int(temp_string[b])
                    temp_array[7] = encrypted_string[count_bits]
                    res = int("".join(str(l) for l in temp_string), 2)
                    S[i][j][k] = res
                    count_bits+=1
                    if(count_bits>=8):
                        break
                if(count_bits>=8):
                        break
            if(count_bits>=8):
                        break
    return S


In [17]:
encoded_shares = encode_shares(shares, N, encrypted_ids)

In [18]:
# saving first k shares
def saveKshares(S, k):
    for i in range(0, k):
        filename = "share_"+str(i+1)+".png"
        cv.imwrite(filename, S[i])
saveKshares(encoded_shares, K)

In [19]:
# Retreiving K shares
def retreive_k_shares(S, k, encrypted_ids):
    R = []
    for i in range(0,k):
        temp = S[i]
        temp_S = rightCircularShift(temp, encrypted_ids[i])
        R.append(temp_S)
    return R

In [20]:
retreived_shares = retreive_k_shares(shares, K, encrypted_ids)


for a in range(K):
        counter_a = 10
        for i in range(len(retreived_shares[a])):
            for j in range(len(retreived_shares[a][i])):
                for t in range(3):
                    print(a, shares[a][i][j][t], retreived_shares[a][i][j][t])
                    counter_a+=1
                    if(counter_a>20):
                        break
                if(counter_a>20):
                        break
            if(counter_a>20):
                        break

0 78 78
0 78 78
0 78 78
0 78 78
0 78 78
0 78 78
0 78 78
0 78 78
0 78 78
0 150 150
0 152 152
1 78 78
1 78 78
1 78 78
1 78 78
1 78 78
1 78 78
1 78 78
1 78 78
1 78 78
1 150 150
1 152 152
2 78 78
2 78 78
2 78 78
2 78 78
2 78 78
2 78 78
2 78 78
2 78 78
2 78 78
2 150 150
2 152 152


In [25]:
# Retreive Secret Image
def getSecretImage(retreived_shares, k, r_shape, rem, key, org):
    ret = np.zeros(r_shape).astype(int)
    for l in range(k):
        for i in range(len(retreived_shares[l])):
            for j in range(len(retreived_shares[l][i])):
                for t in range(3):
                    ret[i][j][t] += retreived_shares[l][i][j][t]
    
    for i in range(len(ret)):
            for j in range(len(ret[i][j])):
                for t in range(3):
                    ret[i][j][t] = ret[i][j][t] + rem[i][j][t]

    counter = 10
    for i in range(len(ret)):
        for j in range(len(ret[i])):
            for t in range(3):
                print(counter, ret[i][j][t], org[i][j][t])
                counter+=1
                if(counter>20):
                    break
            if(counter>20):
                break
        if(counter>20):
            break
    
    secret_image = np.zeros(r_shape).astype(int)
    for i in range(len(ret)):
            for j in range(len(ret[i][j])):
                for t in range(3):
                    secret_image[i][j][t] = ret[i][j][t]^key[i][j][t]
    
    fileName = "retreived_image.png"
    cv.imwrite(fileName, secret_image)

getSecretImage(retreived_shares, K, image.shape, authenticated_image[1], random_key, encrypted_image)
    


10 235 118
11 234 255
12 235 124
13 236 26
14 235 91
15 236 179
16 235 148
17 236 179
18 235 40
19 450 225
20 456 228
